In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from typing import Optional,List
from tqdm.notebook import tqdm
import datetime
import os
import copy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from recs.dataset import session_parallel_dataset
from recs.evaluator import metrics

import tensorflow as tf
from tensorflow import keras as tfk
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd


# Tensorflowによる実装

In [4]:
class SQNGRUModel(tfk.layers.Layer):
    
    def __init__(
        self,
        num_items:int,
        seq_len:Optional[int]=3,
        hidden_dim:Optional[int]=100,
        embed_dim:Optional[int]=100,
        dropout_rate:Optional[float]=0.5,
        name="GRU"
    ):
        super(SQNGRUModel, self).__init__(name=name)
        
        
        self._embedding = tfk.layers.Embedding(num_items, embed_dim, mask_zero=True)
        self._gru = tfk.layers.GRU(
            hidden_dim, 
            dropout=dropout_rate)

        self._dense = tfk.layers.Dense(num_items, activation="softmax")
        self._qvalue_dense = tfk.layers.Dense(num_items, activation=None)
    
    def call(
        self, 
        item_seqs:tf.Tensor, # (batch_size, seq_len)
        training:Optional[bool]=False,
        is_next:Optional[bool]=False
    ):
        x = self._embedding(item_seqs)
        x = self._gru(x, training=training)
        qvalue = self._qvalue_dense(x)
        if is_next:
            return qvalue
        out = self._dense(x)
            
        return out, qvalue
        

In [5]:
class SQNGRU4Rec(tfk.Model):
    
    def __init__(
        self,
        num_items:int,
        seq_len:Optional[int]=3,
        hidden_dim:Optional[int]=100,
        embed_dim:Optional[int]=100,
        dropout_rate:Optional[float]=0.5,
        gamma:Optional[float]=1.,
        k:Optional[int]=20,
        name="SQN-GRUModel"
    ):
        super(SQNGRU4Rec, self).__init__(name=name)
        self._gamma = gamma
        self._num_items = num_items
        self._topk = k
        
        self._gmodel = SQNGRUModel(num_items, seq_len, hidden_dim, embed_dim, dropout_rate, name="SQNGRU")
        self._target_gmodel = SQNGRUModel(num_items, seq_len, hidden_dim, embed_dim, dropout_rate, name="TargetSQNGRU")
        
        self._loss_tracker = tfk.metrics.Mean(name="loss")
        self._tdloss_tracker = tfk.metrics.Mean(name="TD Error")
        self._recall_tracker = tfk.metrics.Recall(name="recall")
            
        dummy_state = tf.zeros((1, seq_len), dtype=tf.int32)
        self._gmodel(dummy_state)
        self._target_gmodel(dummy_state)

    
    def compile(self, g_loss, q_loss, optimizer):
        super(SQNGRU4Rec, self).compile()
        self.g_loss = g_loss
        self.q_loss = q_loss
        self.optimizer = optimizer
    
    def call(self, states):
        x, _ = self._gmodel(states)
        return x
    
    def __train_step(self, data):
        state, action, reward, n_state, done = data
        onehot_act = tf.one_hot(action-1, depth=self._num_items)

        
        with tf.GradientTape() as tape:
            out, qvalue = self._gmodel(state, training=True)
            n_qvalue = self._gmodel(n_state, training=True, is_next=True)
            n_qvalue_ = self._target_gmodel(n_state, training=True, is_next=True)
            
            greedy_a = tf.argmax(n_qvalue, axis=-1)
            onehot_greedy_a = tf.one_hot(greedy_a, depth=self._num_items)
            
            Lq = reward + (1.0 - done) * self._gamma * tf.reduce_sum(n_qvalue_*onehot_greedy_a, axis=-1)
            Lq = tf.stop_gradient(Lq)
            Lq = self.q_loss(Lq, tf.reduce_sum(qvalue*onehot_act,axis=-1))
            
            Ls = self.g_loss(onehot_act, out)
            loss = Lq + Ls
            
            
        grads = tape.gradient(loss, self._gmodel.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self._gmodel.trainable_variables))
        self._loss_tracker.update_state(loss)

        self._tdloss_tracker.update_state(Lq)
        
        return {"loss": self._loss_tracker.result(), "TD Error":self._tdloss_tracker.result()}
    
    def __tar_train_step(self, data):
        state, action, reward, n_state, done = data
        onehot_act = tf.one_hot(action-1, depth=self._num_items)

        
        with tf.GradientTape() as tape:
            out, qvalue = self._target_gmodel(state, training=True)
            n_qvalue = self._target_gmodel(n_state, training=True, is_next=True)
            n_qvalue_ = self._gmodel(n_state, training=True, is_next=True)
            
            greedy_a = tf.argmax(n_qvalue, axis=-1)
            onehot_greedy_a = tf.one_hot(greedy_a, depth=self._num_items)
            
            Lq = reward + (1.0 - done) * self._gamma * tf.reduce_sum(n_qvalue_*onehot_greedy_a, axis=-1)
            Lq = tf.stop_gradient(Lq)
            Lq = self.q_loss(Lq, tf.reduce_sum(qvalue*onehot_act,axis=-1))
            
            Ls = self.g_loss(onehot_act, out)
            loss = Lq + Ls
            
            
        grads = tape.gradient(loss, self._target_gmodel.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self._target_gmodel.trainable_variables))
        self._loss_tracker.update_state(loss)

        self._tdloss_tracker.update_state(Lq)
        
        return {"loss": self._loss_tracker.result(), "TD Error":self._tdloss_tracker.result()}
        
    def train_step(self, data):
        
        if np.random.uniform(0, 1) <= 0.5:
            loss_hist = self.__train_step(data)
        else:
            loss_hist = self.__tar_train_step(data)
        
        return loss_hist   
    
    def test_step(self, data):
        state, target, _, _, _ = data
        target = tf.one_hot(target-1, depth=self._num_items)
        target = tf.cast(target, dtype=tf.int32)

        qvalue = self(state)
        topkitem = tf.math.top_k(qvalue, k=self._topk)[1]
        topkitem = tf.reduce_sum(tf.one_hot(topkitem, depth=self._num_items), axis=1)
        topkitem = tf.cast(topkitem, dtype=tf.int32)
        
        self._recall_tracker.update_state(target, topkitem)
        
        return {"recall":self._recall_tracker.result()}
    
    @property
    def metrics(self):
        return [self._loss_tracker, self._recall_tracker]

In [6]:
dataname="diginetica"
modelname = "SQNGRU4Rec"
default_logdir = "/home/inoue/work/recs/"
log_dir =  os.path.join(default_logdir, "logs/%s/%s/"%(dataname, modelname)+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
train = pickle.load(open(
    "/home/inoue/work/dataset/%s/derived/mdp_train.df"%dataname, "rb"
))

split_ind = int(len(train[0])*0.9)
data = pd.read_pickle("~/work/dataset/%s/derived/train.df"%dataname)
testdata = pd.read_pickle("~/work/dataset/%s/derived/test.df"%dataname)

num_items = max(data.itemId.max(), testdata.itemId.max())+1
emb_dim = 64
hidden_dim = 64
seq_len = train[1].shape[1]
batch_size=500

train_data = tf.data.Dataset.from_tensor_slices(
    (train[1][:split_ind, :],
     train[2][:split_ind],
     train[3][:split_ind],
     train[4][:split_ind, :], 
     train[5][:split_ind].astype(np.float32))
).shuffle(len(train[0][:split_ind])).batch(batch_size)
valid_data = tf.data.Dataset.from_tensor_slices(
    (train[1][split_ind:, :],
     train[2][split_ind:],
     train[3][split_ind:],
     train[4][split_ind:, :], 
     train[5][split_ind:].astype(np.float32))
).shuffle(len(train[0][split_ind:])).batch(batch_size)

In [7]:
model = SQNGRU4Rec(
    num_items, 
    seq_len, 
    hidden_dim, 
    emb_dim, 
    dropout_rate=0.1, gamma=0.5)
model.compile(
    q_loss=tfk.losses.Huber(), 
    g_loss=tfk.losses.CategoricalCrossentropy(),
    optimizer=tfk.optimizers.Adam(learning_rate=0.01)
)
model.build(input_shape=(1,seq_len))

In [8]:
model.fit(
    train_data, 
    epochs=100, 
    validation_data=valid_data,
    validation_freq=1,
    callbacks=[
        tfk.callbacks.TensorBoard(log_dir=log_dir), 
        tfk.callbacks.ModelCheckpoint(
            filepath=os.path.join(default_logdir, "params/%s/checkpoint"%modelname),
            save_weights_only=True,
            monitor="val_recall",
            mode="max",
            save_best_only=True
        ),
        tfk.callbacks.EarlyStopping(
            monitor="val_recall",
            min_delta=1e-4,
            patience=3,
            mode="max",
            verbose=1
        )
    ]
)

Epoch 1/100
646/646 [==============================] - 53s 70ms/step - loss: 9.3911 - TD Error: 0.2805 - val_recall: 0.2365
Epoch 2/100
646/646 [==============================] - 45s 68ms/step - loss: 7.1350 - TD Error: 0.3016 - val_recall: 0.3413
Epoch 3/100
646/646 [==============================] - 44s 67ms/step - loss: 6.0524 - TD Error: 0.3572 - val_recall: 0.3690
Epoch 4/100
646/646 [==============================] - 44s 67ms/step - loss: 5.3290 - TD Error: 0.4013 - val_recall: 0.3733
Epoch 5/100
646/646 [==============================] - 43s 65ms/step - loss: 4.8284 - TD Error: 0.4283 - val_recall: 0.3710
Epoch 6/100
646/646 [==============================] - 43s 64ms/step - loss: 4.4503 - TD Error: 0.4456 - val_recall: 0.3670
Epoch 7/100
646/646 [==============================] - 42s 64ms/step - loss: 4.1650 - TD Error: 0.4561 - val_recall: 0.3592
Epoch 7: early stopping


In [7]:
model = SQNGRU4Rec(
    num_items, 
    seq_len, 
    hidden_dim, 
    emb_dim, 
    dropout_rate=0.1, gamma=0.5)
model.load_weights("/home/inoue/work/recs/params/diginetica/SQNGRU/checkpoint")

In [8]:
test = pickle.load(open(
    "/home/inoue/work/dataset/%s/derived/mdp_test.df"%dataname, "rb"))

test_data = tf.data.Dataset.from_tensor_slices(
    (test[0],test[1],test[2])).shuffle(len(test[0])).batch(batch_size)

In [9]:
k =20
df = pd.DataFrame(columns=["sessionId", "recIds", "choiceId"])
for batch in tqdm(test_data):
    sess, state, target = batch
    pred_score = model(state)
    topkitem = tf.math.top_k(pred_score, k=k)[1].numpy() + 1
    tmp = pd.DataFrame(
        [sess.numpy(), topkitem, target.numpy()]).T
    tmp.columns = ["sessionId", "recIds", "choiceId"]
    df = pd.concat([df, tmp], axis=0)

  0%|          | 0/251 [00:00<?, ?it/s]

In [10]:
for k_ in [5, 10, 15, 20]:
    df["NDCG@%d"%k_] = df[["recIds", "choiceId"]].apply(lambda x: metrics.ndcg_at_k(x[1], x[0], k=k_), axis=1)
    df["Hit@%d"%k_] = df[["recIds", "choiceId"]].apply(lambda x: metrics.hit_at_k(x[1], x[0], k=k_), axis=1)

In [11]:
df.groupby("sessionId").mean().mean()

NDCG@5     0.080413
Hit@5      0.085838
NDCG@10    0.102534
Hit@10     0.133567
NDCG@15    0.115500
Hit@15     0.167548
NDCG@20    0.124714
Hit@20     0.194600
dtype: float64

# GRU4Rec

In [4]:
class GRU4Rec(tfk.Model):
    
    def __init__(
        self,
        num_items:int,
        seq_len:Optional[int]=3,
        hidden_dim:Optional[int]=100,
        embed_dim:Optional[int]=100,
        dropout_rate:Optional[float]=0.5,
        k:Optional[int]=20,
        name="GRU"
    ):
        super(GRU4Rec, self).__init__(name=name)
        self._topk = k
        self._num_items = num_items
        self._embedding = tfk.layers.Embedding(num_items, embed_dim, mask_zero=True)
        self._gru = tfk.layers.GRU(
            hidden_dim, 
            dropout=dropout_rate)

        self._dense = tfk.layers.Dense(num_items, activation="softmax")
        self._recall_tracker = tfk.metrics.Recall(name="recall")
        
    def call(
        self, 
        item_seqs:tf.Tensor,
        training:Optional[bool]=False
    ):
        
        x = self._embedding(item_seqs)
        x = self._gru(x, training=training)
        out = self._dense(x)
        return out
    
    def test_step(self, data):
        state, target = data
        target = tf.one_hot(target, depth=self._num_items)
        target = tf.cast(target, dtype=tf.int32)

        qvalue = self(state)
        topkitem = tf.math.top_k(qvalue, k=self._topk)[1]
        topkitem = tf.reduce_sum(tf.one_hot(topkitem, depth=self._num_items), axis=1)
        topkitem = tf.cast(topkitem, dtype=tf.int32)
        
        self._recall_tracker.update_state(target, topkitem)
        
        return {"recall":self._recall_tracker.result()}

In [5]:
dataname="diginetica"
modelname = "GRU4Rec"
default_logdir = "/home/inoue/work/recs/"
log_dir =  os.path.join(default_logdir, "logs/%s/%s/"%(dataname, modelname)+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
train = pickle.load(open(
    "/home/inoue/work/dataset/%s/derived/mdp_train.df"%dataname, "rb"
))

split_ind = int(len(train[0])*0.9)
data = pd.read_pickle("~/work/dataset/%s/derived/train.df"%dataname)
testdata = pd.read_pickle("~/work/dataset/%s/derived/test.df"%dataname)

num_items = max(data.itemId.max(), testdata.itemId.max())+1
emb_dim = 64
hidden_dim = 64
seq_len = train[1].shape[1]
batch_size=500

train_data = tf.data.Dataset.from_tensor_slices(
    (train[1][:split_ind, :],
     train[2][:split_ind]-1)).shuffle(len(train[0][:split_ind])).batch(batch_size)
valid_data = tf.data.Dataset.from_tensor_slices(
    (train[1][split_ind:, :],
     train[2][split_ind:]-1)
).shuffle(len(train[0][split_ind:])).batch(batch_size)

In [6]:
model = GRU4Rec(num_items, seq_len, hidden_dim, emb_dim, dropout_rate=0.1)
model.compile(
    loss=tfk.losses.SparseCategoricalCrossentropy(),
    optimizer=tfk.optimizers.Adam(learning_rate=0.01))

# model.build(input_shape=(1,seq_len))

In [7]:
model.fit(
    train_data, 
    epochs=100, 
    validation_data=valid_data,
    validation_freq=1,
    callbacks=[
        tfk.callbacks.TensorBoard(log_dir=log_dir), 
        tfk.callbacks.ModelCheckpoint(
            filepath=os.path.join(default_logdir, "params/%s/checkpoint"%modelname),
            save_weights_only=True,
            monitor="val_recall",
            mode="max",
            save_best_only=True
        ),
        tfk.callbacks.EarlyStopping(
            monitor="val_recall",
            min_delta=1e-4,
            patience=3,
            mode="max",
            verbose=1
        )
    ]
)

Epoch 1/100
646/646 [==============================] - 40s 55ms/step - loss: 9.0971 - recall: 0.0000e+00 - val_recall: 0.2430
Epoch 2/100
646/646 [==============================] - 34s 51ms/step - loss: 6.7215 - recall: 0.0000e+00 - val_recall: 0.3374
Epoch 3/100
646/646 [==============================] - 35s 53ms/step - loss: 5.4367 - recall: 0.0000e+00 - val_recall: 0.3508
Epoch 4/100
646/646 [==============================] - 34s 52ms/step - loss: 4.6938 - recall: 0.0000e+00 - val_recall: 0.3441
Epoch 5/100
646/646 [==============================] - 34s 52ms/step - loss: 4.2110 - recall: 0.0000e+00 - val_recall: 0.3393
Epoch 6/100
646/646 [==============================] - 34s 52ms/step - loss: 3.8773 - recall: 0.0000e+00 - val_recall: 0.3337
Epoch 6: early stopping


In [8]:
test = pickle.load(open(
    "/home/inoue/work/dataset/%s/derived/mdp_test.df"%dataname, "rb"))

test_data = tf.data.Dataset.from_tensor_slices(
    (test[0],test[1],test[2]-1)).shuffle(len(test[0])).batch(batch_size)

In [9]:
k =20
df = pd.DataFrame(columns=["sessionId", "recIds", "choiceId"])
for batch in tqdm(test_data):
    sess, state, target = batch
    pred_score = model(state)
    topkitem = tf.math.top_k(pred_score, k=k)[1].numpy()
    tmp = pd.DataFrame(
        [sess.numpy(), topkitem, target.numpy()]).T
    tmp.columns = ["sessionId", "recIds", "choiceId"]
    df = pd.concat([df, tmp], axis=0)

  0%|          | 0/251 [00:00<?, ?it/s]

In [10]:
for k_ in [5, 10, 15, 20]:
    df["NDCG@%d"%k_] = df[["recIds", "choiceId"]].apply(lambda x: metrics.ndcg_at_k(x[1], x[0], k=k_), axis=1)
    df["Hit@%d"%k_] = df[["recIds", "choiceId"]].apply(lambda x: metrics.hit_at_k(x[1], x[0], k=k_), axis=1)

In [14]:
df.groupby("sessionId").mean().mean()

NDCG@5     0.088070
Hit@5      0.093856
NDCG@10    0.112826
Hit@10     0.147244
NDCG@15    0.127746
Hit@15     0.186368
NDCG@20    0.138285
Hit@20     0.217302
dtype: float64